__Decorator__ : Decorators allow you to inject or modify code in functions or classes.

__Function Decorator__ : A function decorator is applied to a function definition by placing it on the line before that function definition begins. Example

@myDecorator
def myFunc():
    print("Hello")

Let create decorator  myDecorator

In [3]:
class myDecorator:
    def __init__(self, f):
        print("Initializing myDecorator")
        f()
    def __call__(self):
        print("I am Inside __call__")

@myDecorator
def my_function():
    print("I am in my_function")

print("Decoration is finished...")

Initializing myDecorator
I am in my_function
Decoration is finished...


In [149]:
my_function()

I am Inside __call__


When a Function() is called after it has been decorated, we get completely different behavior; the my_decorator.__call__() method is called instead of the original code. That’s because the act of decoration replaces the original function object with the result of the decoration. classes we use as decorators must implement `__call__`.

__Another Example__

In [7]:
class hello:
    def __init__(self, f):
        self.f = f
    def __call__(self):
        print("Here i am: " + self.f.__name__ )
        self.f()

In [8]:
@hello
def func1():
    print("I am function 1")

@hello
def func2():
    print("I am Function 2")

In [9]:
func1()
func2()

Here i am: func1
I am function 1
Here i am: func2
I am Function 2


__Decorator Without Argument__ :
If we create a decorator without arguments, the function to be decorated is passed to the constructor, and the `__call__()` method is called whenever the decorated function is invoked. Any arguments for the decorated function are just passed to `__call__()`

In [153]:
class decorator_without_arguments:
    def __init__(self, f):
        self.f = f 
        print("Initializing Decorator for: " + self.f.__name__)
    def __call__(self, a, b):
        print("Inside Function: " + self.f.__name__)
        self.f(a, b)

In [154]:
@decorator_without_arguments
def func1(a, b):
    print("Func Args: {}, {}".format(a,b))

@decorator_without_arguments
def func2(a, b):
    print("Func Args: {}, {}".format(a,b))

Initializing Decorator for: func1
Initializing Decorator for: func2


In [155]:
func1(1,2)
func2(3,4)

Inside Function: func1
Func Args: 1, 2
Inside Function: func2
Func Args: 3, 4


__Decorator With Argument__:
The decorator mechanism behaves quite differently when you pass arguments to the decorator.

Let’s modify the above example to see what happens when we add arguments to the decorator:


In [156]:
class decorator_with_arguments:
    def __init__(self, arg1, arg2):
        self.arg1 = arg1
        self.arg2 = arg2
    
    def __call__(self ,f):
        print("Inside __call__(): " + f.__name__)
        print("Decorator Args: {}, {}".format(self.arg1, self.arg2))
        def inner(a,b):
            f(a,b)
        return inner

In [157]:
@decorator_with_arguments("a", "b")
def func1(a, b):
    print("Args: {}, {}".format(a,b))

@decorator_with_arguments("c", "d")
def func2(a, b):
    print("Args: {}, {}".format(a,b))

Inside __call__(): func1
Decorator Args: a, b
Inside __call__(): func2
Decorator Args: c, d


Now the process of decoration calls the constructor and then immediately invokes `__call__()`, which can only take a single argument (the function object) and must return the decorated function. 

In [158]:
func1(1,2)
func2(3,4)

Args: 1, 2
Args: 3, 4


__Using Function as Decorator__

In [159]:
def decoratorHello(func):
    print("Initializing decorator for: " + func.__name__)
    def inner(a,b):
        func(a,b)
    return inner

In [160]:
@decoratorHello
def func1(a,b):
    print("I am func1, Args: {}, {}".format(a,b))

@decoratorHello
def func2(a,b):
    print("I am func2, Args: {}, {}".format(a,b))

Initializing decorator for: func1
Initializing decorator for: func2


In [161]:
func1(1,2)
func2(3,4)

I am func1, Args: 1, 2
I am func2, Args: 3, 4


__Decorator Function Without Argument__ : The above example is the one,  Any arguments for the decorated function are just passed to `inner()`

__Decorated Function With Argument__: 

In [162]:
def decoratorHello(arg1, arg2):
    def wrap(func):
        print("Initializing decorator for: " + func.__name__)
        print("Decorator Arg: {}, {}".format(arg1, arg2))
        def inner(a,b):
            func(a,b)
        return inner
    return wrap

In [163]:
@decoratorHello("a" ,"b")
def func1(a,b):
    print("Args: {}, {}".format(a,b))

@decoratorHello("c", "d")
def func2(a,b):
    print("Args: {}, {}".format(a,b))

Initializing decorator for: func1
Decorator Arg: a, b
Initializing decorator for: func2
Decorator Arg: c, d


In [164]:
func1(1,2)
func2(2,3)

Args: 1, 2
Args: 2, 3



The return value of the decorator function must be a function used to wrap the function to be decorated. That is, Python will take the returned function and call it at decoration time, passing the function to be decorated. That’s why we have three levels of functions; the inner one is the actual replacement function.

Because of closures, wrapped_f() has access to the decorator arguments arg1, arg2 and arg3, without having to explicitly store them as in the class version. However, this is a case where I find “explicit is better than implicit,” so even though the function version is more succinct I find the class version easier to understand and thus to modify and maintain.

<pre>📝 Decorator Metadata Notes (Python)
THE REAL RULE (the only rule you should remember)

If your decorator returns a function → use @wraps.
If your decorator returns a class instance → use update_wrapper.</pre>

In [2]:
#Broken decorator (missing metadata)
from functools import wraps, update_wrapper

# BROKEN DECORATOR — NO @wraps
def log(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

@log
def add(a, b):
    """Add two numbers"""
    return a + b

print(add.__name__)   # WRONG: wrapper
print(add.__doc__)    # WRONG: None


wrapper
None


In [3]:
def log(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

@log
def add(a, b):
    """Add two numbers"""
    return a + b

print(add.__name__)   # WRONG: wrapper
print(add.__doc__)    # WRONG: None

add
Add two numbers


In [ ]:
from functools import update_wrapper

class DivByZero:
    def __init__(self, func):
        # decorator returns an OBJECT, so fix metadata on self
        update_wrapper(self, func)
        self.func = func
    
    def __call__(self, a, b):
        if b == 0:
            raise ZeroDivisionError("b cannot be zero")
        return self.func(a, b)

"""
tmp = DivByZero(div)     # Decorator return → OBJECT
div = tmp                # func replaced with object
"""
@DivByZero
def div(a, b):
    """Divide two numbers"""
    return a / b

print(div.__name__)   # div
print(div.__doc__)    # Divide two numbers
print(div(10, 2))     # 5.0


# @DivByZero → div = DivByZero(div)
# The decorator returns an instance, so metadata must be copied onto that instance.

In [4]:
# Example 2: Decorator returns a function
# use @wraps

from functools import wraps

class DecoratorWithArgument:
    def __init__(self, a, b):
        self.a = a
        self.b = b
    
    def __call__(self, func):
        @wraps(func)   # decorator returns a FUNCTION → use wraps
        def inner(x, y):
            print("Before calling:", func.__name__)
            return func(x, y)
        return inner
    
"""
tmp = DecorotarWithArgument("Coco", "Mini")
func1 = tmp(func1)      # Decorator return → FUNCTION (inner)
"""
@DecoratorWithArgument("Coco", "Mini")
def func1(a, b):
    """Demo function"""
    print("Inside function:", a, b)

print(func1.__name__)  # func1
print(func1.__doc__)   # Demo function
func1(10, 20)


func1
Demo function
Before calling: func1
Inside function: 10 20
